# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

In [1]:
import string
from pathlib import Path
from typing import Iterable

import nltk
import numpy as np
import pandas as pd
import pymorphy2
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [2]:
DATA_DIR = Path('data/')

## Задачи для совместного разбора

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [3]:
with open(DATA_DIR.joinpath('litw-win.txt'), encoding='cp1251') as f:
    words = [line.strip().split()[-1].lower() for line in f]

words[-5:]

['высокопревосходительства',
 'попреблагорассмотрительст',
 'попреблагорассмотрительствующемуся',
 'убегающих',
 'уменьшившейся']

In [4]:
text = ('с велечайшим усилием выбравшись из потока убегающих людей Кутузов со '
        'свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий')

In [5]:
def find_replacements(text: str, dictionary: Iterable) -> dict[str, tuple[int, list[str]]]:
    dictionary = set(dictionary)
    replacements = {}
    for word in text.lower().split():
        if word not in dictionary:
            sorted_edit_distances = sorted(
                map(lambda x: (nltk.edit_distance(word, x), x), dictionary),
                key=lambda x: x[0]
            )
            min_edit_distance = sorted_edit_distances[0][0]
            replacement_list = []
            for edit_distance, dict_word in sorted_edit_distances:
                if edit_distance > min_edit_distance:
                    break
                replacement_list.append(dict_word)
            replacements[word] = (min_edit_distance, replacement_list)

    return replacements

In [ ]:
replacements = find_replacements(text, words)
replacements

In [ ]:
fixed_text = text
for word, repl in replacements.items():
    fixed_text = fixed_text.replace(word, repl[1][0])
fixed_text

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
task1_text = (
    'Считайте слова из файла `litw-win.txt` и запишите их в список `words`. '
    'В заданном предложении исправьте все опечатки, заменив слова с опечатками '
    'на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. '
    'Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`.'
)
stemmer = nltk.SnowballStemmer('russian')
lemmer = pymorphy2.MorphAnalyzer()

In [ ]:
pd.DataFrame(
    data=[
        [word, stemmer.stem(word), lemmer.parse(word)[0].normal_form]
        for word in nltk.word_tokenize(task1_text)
        if word not in string.punctuation
    ],
    columns=['word', 'stemma', 'lemma']
).sample(5)

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [ ]:
CountVectorizer().fit_transform(nltk.sent_tokenize(task1_text)).toarray()

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
descriptions_df = pd.read_csv(DATA_DIR.joinpath('preprocessed_descriptions.csv'), sep=',')
descriptions_df = descriptions_df.rename(columns={'preprocessed_descriptions': 'description'})
descriptions_df.info()
descriptions_df.head()

In [ ]:
descriptions = descriptions_df['description'].dropna()
words = list({word for text in descriptions for word in nltk.word_tokenize(text)})
len(words)

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
rng = np.random.default_rng()
random_pairs = rng.choice(words, size=(5, 2)).tolist()
list(map(lambda x: (x, nltk.edit_distance(x[0], x[1])), random_pairs))

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def similarity(word: str, words: list[str], k: int) -> list[str]:
    return sorted(words, key=lambda x: nltk.edit_distance(word, x))[:k]

In [ ]:
similarity('hello', words, 5)

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для лемматизации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
stemmer = nltk.SnowballStemmer('english')
lemmatizer = nltk.WordNetLemmatizer()

In [ ]:
df = pd.DataFrame(
    data={
        'stemmed_word': map(stemmer.stem, words),
        'normalized_word': map(lemmatizer.lemmatize, words),
    },
    index=pd.Series(words, name='words')
)
df.head()

In [ ]:
print(f'Всего слов: {len(df)}')
print(f'Основ отличается: {len(set(df.index) - set(df["stemmed_word"]))}')
print(f'Лемм отличается: {len(set(df.index) - set(df["normalized_word"]))}')

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
all_words = [word for text in descriptions for word in nltk.word_tokenize(text)]
words_without_stopwords = [word for word in all_words if word not in stop_words]

In [ ]:
n_words = len(all_words)
n_words_without_stopwords = len(words_without_stopwords)
print(f'Слов всего: {n_words}')
print(f'Слов всего после удаления стоп-слов: {n_words_without_stopwords}')
print(f'Доля стоп-слов {(n_words - n_words_without_stopwords) / n_words:.2%}')

In [ ]:
print(f'Топ-10 по частоте (до): \n{nltk.FreqDist(all_words).most_common(10)}')
print(f'Топ-10 по частоте (после): \n{nltk.FreqDist(words_without_stopwords).most_common(10)}')

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
descriptions_sample = descriptions_df.dropna().sample(5)
descriptions_sample

In [ ]:
tfid_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
tfid_vectorizer.fit(descriptions_sample['description'])
descriptions_sample['vector'] = descriptions_sample['description'].apply(
    lambda x: tfid_vectorizer.transform([x]).toarray()
)
print(descriptions_sample.to_string())

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
name_values = [name[:8] + '...' for name in descriptions_sample['name'].array]
vector_values = [v.reshape(-1) for v in descriptions_sample['vector'].array]

In [ ]:
cosine_distances_df = pd.DataFrame(
    data=cosine_distances(vector_values, vector_values),
    index=name_values,
    columns=name_values,
)
cosine_distances_df

In [ ]:
cosine_similarity_df = pd.DataFrame(
    data=cosine_similarity(vector_values, vector_values),
    index=name_values,
    columns=name_values,
)
cosine_similarity_df

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [ ]:
eps = 1e-6
similarity_top = cosine_similarity_df.to_numpy()
np.sort(similarity_top[(0 + eps < similarity_top) & (similarity_top < 1 - eps)])[::-1]

In [ ]:
# Два различных рецепта тем более похожи,
# чем меньше косинусное расстояние между ними
# (или чем больше косинусная схожесть: 1 - cosine)
